# Project 3: Smart Beta Portfolio and Portfolio Optimization

## Overview


Smart beta has a broad meaning, but we can say in practice that when we use the universe of stocks from an index, and then apply some weighting scheme other than market cap weighting, it can be considered a type of smart beta fund.  A Smart Beta portfolio generally gives investors exposure or "beta" to one or more types of market characteristics (or factors) that are believed to predict prices while giving investors a diversified broad exposure to a particular market. Smart Beta portfolios generally target momentum, earnings quality, low volatility, and dividends or some combination. Smart Beta Portfolios are generally rebalanced infrequently and follow relatively simple rules or algorithms that are passively managed.  Model changes to these types of funds are also rare requiring prospectus filings with US Security and Exchange Commission in the case of US focused mutual funds or ETFs.. Smart Beta portfolios are generally long-only, they do not short stocks.

In contrast, a purely alpha-focused quantitative fund may use multiple models or algorithms to create a portfolio. The portfolio manager retains discretion in upgrading or changing the types of models and how often to rebalance the portfolio in attempt to maximize performance in comparison to a stock benchmark.  Managers may have discretion to short stocks in portfolios.

Imagine you're a portfolio manager, and wish to try out some different portfolio weighting methods.

One way to design portfolio is to look at certain accounting measures (fundamentals) that, based on past trends, indicate stocks that produce better results.  


For instance, you may start with a hypothesis that dividend-issuing stocks tend to perform better than stocks that do not. This may not always be true of all companies; for instance, Apple does not issue dividends, but has had good historical performance.  The hypothesis about dividend-paying stocks may go something like this: 

Companies that regularly issue dividends may also be more prudent in allocating their available cash, and may indicate that they are more conscious of prioritizing shareholder interests.  For example, a CEO may decide to reinvest cash into pet projects that produce low returns.  Or, the CEO may do some analysis, identify that reinvesting within the company produces lower returns compared to a diversified portfolio, and so decide that shareholders would be better served if they were given the cash (in the form of dividends).  So according to this hypothesis, dividends may be both a proxy for how the company is doing (in terms of earnings and cash flow), but also a signal that the company acts in the best interest of its shareholders.  Of course, it's important to test whether this works in practice.


You may also have another hypothesis, with which you wish to design a portfolio that can then be made into an ETF.  You may find that investors may wish to invest in passive beta funds, but wish to have less risk exposure (less volatility) in their investments.  The goal of having a low volatility fund that still produces returns similar to an index may be appealing to investors who have a shorter investment time horizon, and so are more risk averse.

So the objective of your proposed portfolio is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  Also, if this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

Smart Beta ETFs can be designed with both of these two general methods (among others): alternative weighting and minimum volatility ETF.


## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.
### Install Packages

In [6]:
import sys
#!{sys.executable} -m pip install --user -r requirements.txt

### Load Packages

In [7]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
For this universe of stocks, we'll be selecting large dollar volume stocks. We're using this universe, since it is highly liquid.

In [8]:
df = pd.read_csv('./data/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

In [9]:
close.head()

ticker,AAL,AAPL,ABBV,ABT,AGN,AIG,AMAT,AMGN,AMZN,APC,...,USB,UTX,V,VLO,VZ,WBA,WFC,WMT,WYNN,XOM
date,,,,,,,,,,,,,,,,,,,,,
2013-07-01,16.17609308,53.10917319,34.92447839,31.42538772,122.62751990,41.55339742,13.63297558,86.80333359,282.10000000,82.90947737,...,32.32004572,84.38110191,44.66007989,29.28427559,40.07272093,41.02191970,35.91510532,65.48939807,112.18880046,76.32080247
2013-07-02,15.81983388,54.31224742,35.42807578,31.27288084,121.05361758,41.36908428,13.63757665,85.53008744,283.73000000,82.89037160,...,32.31114458,83.54388177,44.66490227,28.67721470,40.28756674,41.13293978,35.79353582,65.59475707,110.19759336,76.60816761
2013-07-03,16.12794994,54.61204262,35.44486235,30.72565028,121.21003024,40.75163526,13.80321523,85.30749196,284.03000000,83.04321774,...,32.35565029,84.37219531,45.09891665,28.80546700,40.58994231,40.81838289,35.79353582,65.63865665,110.16265990,76.65042719
2013-07-05,16.21460758,54.17338125,35.85613355,31.32670680,123.53666845,41.64555399,14.00566239,86.93689088,285.88000000,84.42838587,...,32.72059710,85.97538281,46.00311328,28.94226945,40.82070262,40.93865464,36.53163639,66.03375290,110.98359616,77.39419581
2013-07-08,16.31089385,53.86579916,36.66188936,31.76628544,123.65397794,41.94967067,13.93204524,87.41769712,290.59000000,84.35196280,...,32.97873021,86.37617969,45.35450290,29.52367989,41.12705682,41.93783537,37.19158513,67.35074039,110.89626252,77.96892611


### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [126]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio
In Part 1 of this project, you'll build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF.  You'll compare this portfolio to a market cap weighted index to see how well it performs. 

Note that in practice, you'll probably get the index weights from a data vendor (such as companies that create indices, like MSCI, FTSE, Standard and Poor's), but for this exercise we will simulate a market cap weighted index.

## Index Weights
The index we'll be using is based on large dollar volume stocks. Implement `generate_dollar_volume_weights` to generate the weights for this index. For each date, generate the weights based on dollar volume traded for that date. For example, assume the following is close prices and volume data:
```
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...
```
The weights created from the function `generate_dollar_volume_weights` should be the following:
```
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...
```

In [12]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function
    dollar_volume = close * volume
    row_sum = dollar_volume.sum(axis=1)
    dollar_volume_weights = dollar_volume.div(row_sum, axis=0)
    return dollar_volume_weights

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [13]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

## Portfolio Weights
Now that we have the index weights, let's choose the portfolio weights based on dividend. You would normally calculate the weights based on trailing dividend yield, but we'll simplify this by just calculating the total dividend yield over time.

Implement `calculate_dividend_weights` to return the weights for each stock based on its total dividend yield over time. This is similar to generating the weight for the index, but it's using dividend data instead.
For example, assume the following is `dividends` data:
```
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...
```
The weights created from the function `calculate_dividend_weights` should be the following:
```
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...
```

In [14]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    dividends_cumsum = dividends.cumsum()
    dividends_cumsum_row_sum = dividends_cumsum.sum(axis=1)
    dividend_weights = dividends_cumsum.div(dividends_cumsum_row_sum, axis=0)
    return dividend_weights

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

Tests Passed


### View Data
Just like the index weights, let's generate the ETF weights and view them using a heatmap.

In [15]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

## Returns
Implement `generate_returns` to generate returns data for all the stocks and dates from price data. You might notice we're implementing returns and not log returns. Since we're not dealing with volatility, we don't have to use log returns.

In [16]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
    return prices.pct_change()

project_tests.test_generate_returns(generate_returns)

Tests Passed


### View Data
Let's generate the closing returns using `generate_returns` and view them using a heatmap.

In [17]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF. Implement `generate_weighted_returns` to create weighted returns using the returns and weights.

In [18]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function
    return returns * weights

project_tests.test_generate_weighted_returns(generate_weighted_returns)

Tests Passed


### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [19]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

## Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [116]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function
    cumulative_returns = (returns.sum(axis=1) + 1).cumprod()
    print((returns.sum(axis=1, skipna=True) + 1).cumprod())
    return cumulative_returns

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

1999-05-12           nan
1999-05-13    5.93647782
1999-05-14   -0.57128454
1999-05-15   -0.68260542
dtype: float64
Tests Passed


### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [117]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

date
2013-07-01          nan
2013-07-02   1.00272899
2013-07-03   1.00477996
2013-07-05   1.01417607
2013-07-08   1.01866796
2013-07-09   1.02794398
2013-07-10   1.02673591
2013-07-11   1.04235354
2013-07-12   1.04239359
2013-07-15   1.04673409
2013-07-16   1.04442747
2013-07-17   1.04980276
2013-07-18   1.05326567
2013-07-19   1.04382419
2013-07-22   1.04747372
2013-07-23   1.04218381
2013-07-24   1.04721663
2013-07-25   1.11356958
2013-07-26   1.11843867
2013-07-29   1.12063868
2013-07-30   1.13058803
2013-07-31   1.12484814
2013-08-01   1.14377361
2013-08-02   1.15023611
2013-08-05   1.15462769
2013-08-06   1.14550337
2013-08-07   1.14416027
2013-08-08   1.14957642
2013-08-09   1.14400095
2013-08-12   1.14734497
                ...    
2017-05-19   2.72018106
2017-05-22   2.73495799
2017-05-23   2.74178792
2017-05-24   2.74944367
2017-05-25   2.76136288
2017-05-26   2.76915666
2017-05-30   2.76883914
2017-05-31   2.76160993
2017-06-01   2.78006084
2017-06-02   2.80401577
2017-06-05 

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [118]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function
    return np.sqrt(252) * np.std((etf_returns_by_date - benchmark_returns_by_date), ddof=1)

project_tests.test_tracking_error(tracking_error)

Tests Passed


### View Data
Let's generate the tracking error using `tracking_error`.

In [119]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.10207614832007529


# Part 2: Portfolio Optimization

Now, let's create a second portfolio.  We'll still reuse the market cap weighted index, but this will be independent of the dividend-weighted portfolio that we created in part 1.

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.


## Covariance
Implement `get_covariance_returns` to calculate the covariance of the `returns`. We'll use this to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [120]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function

    returns_2d_array = returns.fillna(0).transpose().values
    returns_covariance = np.cov(returns_2d_array)

    return returns_covariance

project_tests.test_get_covariance_returns(get_covariance_returns)

Tests Passed


### View Data
Let's look at the covariance generated from `get_covariance_returns`.

In [121]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?



### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [122]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]
    

    # print(covariance_returns)
    
    #TODO: Implement function
    m = len(covariance_returns)
    x = cvx.Variable(m)
    portfolio_variance = cvx.quad_form(x, covariance_returns)
    distance_to_index = cvx.norm(x - index_weights)
    objective = cvx.Minimize(portfolio_variance + scale + distance_to_index)
    constraints = [x >= 0, sum(x) == 1]
    p = cvx.Problem(objective, constraints)
    p.solve()
    return x.value

project_tests.test_get_optimal_weights(get_optimal_weights)

Tests Passed


## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [123]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [124]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

date
2013-07-01          nan
2013-07-02   1.00123995
2013-07-03   1.00272304
2013-07-05   1.01206939
2013-07-08   1.01635807
2013-07-09   1.02329417
2013-07-10   1.02249292
2013-07-11   1.03620410
2013-07-12   1.04190291
2013-07-15   1.04431573
2013-07-16   1.04325334
2013-07-17   1.04878233
2013-07-18   1.05023152
2013-07-19   1.04870890
2013-07-22   1.05183749
2013-07-23   1.04819997
2013-07-24   1.04603238
2013-07-25   1.06123480
2013-07-26   1.06193263
2013-07-29   1.05979166
2013-07-30   1.06341554
2013-07-31   1.06373927
2013-08-01   1.07984481
2013-08-02   1.08329213
2013-08-05   1.08540158
2013-08-06   1.07725823
2013-08-07   1.07332500
2013-08-08   1.07626840
2013-08-09   1.07121810
2013-08-12   1.07074433
                ...    
2017-05-19   2.18845645
2017-05-22   2.19977526
2017-05-23   2.20663025
2017-05-24   2.21438915
2017-05-25   2.22464527
2017-05-26   2.23014436
2017-05-30   2.23063661
2017-05-31   2.22617514
2017-06-01   2.23788649
2017-06-02   2.25133536
2017-06-05 

Optimized ETF Tracking Error: 0.05795012629906411


## Rebalance Portfolio Over Time
The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [125]:
from datetime import timedelta


def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0

    # TODO: Implement function
    all_rebalance_weights = []

    for r in range(chunk_size, len(returns), shift_size):

        weights = pd.Series(index_weights.iloc[r-1])
        covariance_returns = get_covariance_returns(returns[(r - chunk_size):r])
        optimal_weights = get_optimal_weights(covariance_returns, weights)
        print(optimal_weights)
        all_rebalance_weights.append(optimal_weights)

    return all_rebalance_weights


project_tests.test_rebalance_portfolio(rebalance_portfolio)

#np.array([0.29341237, 0.41378419, 0.29280344])
#np.array([0.29654088, 0.40731481, 0.29614432])
#np.array([0.29868214, 0.40308791, 0.29822995])
#np.array([0.30100044, 0.39839644, 0.30060312])

[0.29341879 0.4137734  0.29280781]
[0.29652998 0.40729881 0.2961712 ]
[0.2986566  0.40307303 0.29827037]
[0.30099716 0.39840216 0.30060067]


AssertionError: Wrong value for rebalance_portfolio.

INPUT index_weights:
                  TTN       EEWR       JBEX
2013-01-06 0.00449404 0.11586048 0.00359727
2013-01-07 0.00403487 0.12534048 0.00344280
2013-01-08 0.00423485 0.12854258 0.00347404
2013-01-09 0.00395679 0.12434660 0.00335064
2013-01-10 0.00368729 0.11750295 0.00333929
2013-01-11 0.00369562 0.11447422 0.00325973
2013-01-12 0.00379612 0.11088075 0.00317340
2013-01-13 0.00366501 0.10806014 0.00314648
2013-01-14 0.00361268 0.10376514 0.00323257
2013-01-15 0.00358844 0.10097531 0.00319009
2013-01-16 0.00362045 0.09791232 0.00318071

INPUT chunk_size:
4

INPUT returns:
                   TTN        EEWR        JBEX
2013-01-06         nan         nan         nan
2013-01-07 -0.02202381  0.02265285  0.01441961
2013-01-08  0.01947657  0.00551985  0.00047382
2013-01-09  0.00537313 -0.00803232  0.01160313
2013-01-10  0.00593824 -0.00567773  0.02247191
2013-01-11  0.02479339  0.01758824 -0.00824176
2013-01-12 -0.01094470 -0.00383

Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [30]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

[0.01043134 0.06006002 0.01145332 0.00460319 0.00899074 0.00659741
 0.00588434 0.00543873 0.01913355 0.00440472 0.00117287 0.00498903
 0.00982399 0.02170975 0.00656725 0.00762632 0.01871608 0.00315683
 0.01490108 0.00299942 0.00180181 0.01017089 0.00302964 0.00757446
 0.00280221 0.00379319 0.00894089 0.00609675 0.01479801 0.00855455
 0.01117655 0.00342065 0.00574597 0.0052358  0.0066932  0.06960769
 0.00447415 0.00371878 0.00759784 0.01160758 0.01144284 0.00780362
 0.0217761  0.02548132 0.00738601 0.01097032 0.0083364  0.00317682
 0.0111048  0.01594194 0.01052272 0.01329633 0.00202779 0.01358219
 0.00451156 0.00311991 0.0050552  0.00628738 0.00545589 0.00816001
 0.01323018 0.00557249 0.01257088 0.00812513 0.00680572 0.01944452
 0.01416098 0.0194583  0.00839333 0.00246675 0.0137056  0.00461931
 0.00608005 0.01084456 0.01092437 0.01692651 0.0077317  0.0113174
 0.00577372 0.00556129 0.03248384 0.00983902 0.00390546 0.00569
 0.00331695 0.00602821 0.00490272 0.00619657 0.00309221 0.00749353

[0.00639348 0.10950969 0.00933782 0.00436677 0.00481393 0.00590568
 0.00875693 0.00769789 0.01685741 0.0071158  0.01237878 0.00576433
 0.00890182 0.01952678 0.00567496 0.00471818 0.01147452 0.01095917
 0.00700347 0.00678017 0.00362661 0.00884257 0.00314553 0.00631869
 0.00289964 0.00848896 0.01010521 0.00456968 0.01000898 0.00642108
 0.0090926  0.00484661 0.00493783 0.00694218 0.00556897 0.05030044
 0.00408125 0.0045128  0.00826949 0.01278599 0.0288528  0.0041964
 0.01583754 0.01800632 0.01117881 0.00854731 0.01688973 0.00262847
 0.01252608 0.02593121 0.01268708 0.01226323 0.01228684 0.00720334
 0.00497498 0.00616333 0.00443283 0.00611278 0.00996706 0.00439679
 0.00711682 0.00343554 0.00844791 0.00776214 0.00403691 0.02297344
 0.01129836 0.01721003 0.00618668 0.00250242 0.00902832 0.00536779
 0.00772596 0.01102607 0.0092586  0.00471827 0.00504897 0.01111278
 0.00472123 0.00437    0.01170829 0.00936915 0.00544269 0.00518293
 0.00385229 0.00499929 0.0068796  0.00480102 0.00522239 0.00333

[0.00626393 0.09759007 0.00840188 0.00321649 0.01060793 0.0088736
 0.00446244 0.01565523 0.01809636 0.00913393 0.00235614 0.00579178
 0.00807661 0.01912679 0.00646454 0.01146827 0.01141412 0.00721641
 0.00544482 0.0102835  0.00255328 0.01120614 0.00356661 0.01105586
 0.0042595  0.00442714 0.00649746 0.00763798 0.01884911 0.00750933
 0.00862951 0.00264504 0.01162248 0.00431971 0.00660605 0.05490088
 0.00917949 0.00459453 0.00625109 0.01454189 0.02848963 0.00483265
 0.01396503 0.01588586 0.00763159 0.01104622 0.0123237  0.00496189
 0.01228619 0.01611321 0.01252966 0.01196787 0.0082867  0.0089371
 0.00378577 0.00359695 0.00527036 0.00996728 0.00622155 0.00503106
 0.00510159 0.00410501 0.00664073 0.00969626 0.00474006 0.01827767
 0.01201917 0.017827   0.00396644 0.0027021  0.00696679 0.00698018
 0.00679279 0.01044646 0.01110582 0.00463149 0.00853699 0.01095235
 0.00616419 0.00692777 0.0123461  0.01176071 0.00270916 0.01049179
 0.0057863  0.00474152 0.00652999 0.00760221 0.00619586 0.005393

[0.01384761 0.1035144  0.00555535 0.00343208 0.00847953 0.00637601
 0.0039992  0.00946569 0.01540223 0.00754476 0.00347726 0.00487645
 0.00772305 0.02371109 0.0065523  0.00567489 0.01283727 0.00638687
 0.00431008 0.00926436 0.00144894 0.01617245 0.00500045 0.0088721
 0.00454025 0.00463492 0.01274134 0.00737084 0.01500466 0.00873223
 0.01044523 0.00315109 0.00720206 0.00540925 0.005489   0.03770936
 0.00676287 0.00347839 0.00599563 0.01532493 0.02916956 0.00892483
 0.01741601 0.0158244  0.00698602 0.010838   0.01367083 0.00390315
 0.01364994 0.0161124  0.01420446 0.02546191 0.00742793 0.00850015
 0.00377836 0.00301155 0.00680282 0.00579648 0.00936415 0.00413504
 0.00767721 0.00529802 0.00595822 0.00916807 0.00567647 0.02218189
 0.00904674 0.0101547  0.00522546 0.0019494  0.01365027 0.00593153
 0.0076769  0.0098715  0.01121093 0.00624328 0.00619887 0.01229821
 0.00751852 0.0071303  0.0100323  0.0163223  0.00522606 0.00489545
 0.00479982 0.00718088 0.00533908 0.00624314 0.00336512 0.00568

[0.00953368 0.1037459  0.00586043 0.00545831 0.01766637 0.00670895
 0.00740335 0.00906722 0.01462101 0.00925135 0.006004   0.00756037
 0.0115152  0.01924967 0.00915092 0.00662683 0.01773507 0.00548298
 0.01126331 0.01108296 0.00516893 0.01164788 0.0047427  0.00835826
 0.00512211 0.00985557 0.01714128 0.00550593 0.01203003 0.00855317
 0.01013613 0.00264204 0.00823914 0.00392129 0.00640009 0.03061724
 0.00553635 0.00467703 0.0062452  0.01349131 0.01305308 0.00646317
 0.0215047  0.02568395 0.00539659 0.00939588 0.0113997  0.00549477
 0.01490311 0.01394271 0.01217386 0.01308742 0.00703845 0.00921847
 0.00441498 0.00341833 0.00652451 0.00886796 0.01995049 0.00431106
 0.00679926 0.00430872 0.00393751 0.00897675 0.0042838  0.02297043
 0.01021111 0.01238621 0.00928257 0.00293767 0.0082287  0.00526782
 0.00572549 0.01168084 0.00985348 0.00401672 0.00311554 0.01027593
 0.00455341 0.00629076 0.01061279 0.01122606 0.00753359 0.0041498
 0.004472   0.00598219 0.00566474 0.00608131 0.00379329 0.00345

...

[0.01003075 0.07027076 0.00609901 0.00456343 0.01899223 0.00618491
 0.00815799 0.00668158 0.04505064 0.00422608 0.00638835 0.00436276
 0.00793726 0.02129222 0.00749476 0.01271759 0.01849566 0.00826975
 0.00240346 0.00668407 0.00578789 0.00792267 0.00626448 0.0053405
 0.00446333 0.00829948 0.01214736 0.00933334 0.01304834 0.01645117
 0.00895842 0.00597953 0.00374254 0.00396612 0.0075158  0.03849802
 0.00808627 0.00423749 0.00370514 0.01354834 0.01292502 0.00625011
 0.01867037 0.01934386 0.00781059 0.00589306 0.00780026 0.00939242
 0.0073429  0.01049087 0.01466037 0.02050533 0.00759184 0.01221134
 0.00478429 0.00435657 0.005322   0.00640202 0.00865023 0.00420012
 0.00695276 0.00505577 0.00383273 0.01512815 0.00748417 0.02438789
 0.00846674 0.01465321 0.00753168 0.01112693 0.00921259 0.00547951
 0.00633256 0.01301658 0.01636427 0.00748743 0.00360492 0.00782594
 0.00351589 0.0062595  0.00635104 0.01124149 0.00367483 0.00548347
 0.00444194 0.00760965 0.00787317 0.00705322 0.00376864 0.00472

[0.00722484 0.05728831 0.00620049 0.00443251 0.01513867 0.00625088
 0.00514059 0.00954524 0.02842744 0.00459451 0.01952605 0.00279719
 0.00610649 0.04408449 0.02484339 0.00959881 0.02001966 0.0090062
 0.00520746 0.00609216 0.00549676 0.00864305 0.00712546 0.00450914
 0.00590556 0.0054612  0.00906711 0.00775994 0.00944262 0.00639879
 0.01881191 0.00607768 0.00638804 0.00803204 0.00812959 0.03128441
 0.00591225 0.00568177 0.01722023 0.01401938 0.01028188 0.01404672
 0.0215233  0.02307315 0.01938059 0.00475549 0.00998549 0.00492094
 0.00739272 0.00839587 0.01239525 0.01794732 0.00372261 0.01680541
 0.00424519 0.0057855  0.00603568 0.00392993 0.00458449 0.00379948
 0.00755544 0.00691771 0.00368945 0.00912757 0.00770593 0.02459501
 0.00735717 0.00864561 0.00445812 0.01306878 0.00664364 0.00449634
 0.00747546 0.01315087 0.00963486 0.00539129 0.0033697  0.00537528
 0.00720355 0.0060564  0.00586774 0.00957321 0.00442332 0.00714
 0.00633732 0.00579266 0.0093968  0.00623412 0.0054617  0.00645003

[0.00320893 0.04940642 0.00517317 0.00463346 0.0258782  0.00333094
 0.00510523 0.00937865 0.03902811 0.00462035 0.00702391 0.00281747
 0.00805424 0.03772085 0.00860773 0.01018147 0.0171184  0.00612807
 0.00373923 0.00631428 0.00533243 0.00860193 0.00722053 0.00687423
 0.00555433 0.00345183 0.01012917 0.0084263  0.00910372 0.00414448
 0.02433875 0.00946164 0.00558599 0.00268595 0.00504191 0.05005814
 0.00779006 0.00295189 0.00388052 0.01499427 0.05491922 0.00930817
 0.01556779 0.01800357 0.01066186 0.00888264 0.00640017 0.00353027
 0.00812256 0.01220218 0.00957603 0.01936933 0.00410291 0.00945434
 0.0035277  0.00401065 0.0047509  0.00527818 0.00566904 0.01102558
 0.00446693 0.00549433 0.00261205 0.00603991 0.00755916 0.01851402
 0.00869531 0.01660654 0.00817335 0.02209636 0.00580539 0.00588942
 0.00573957 0.00978543 0.00963543 0.00783784 0.01407053 0.00767705
 0.0051602  0.01048453 0.0069438  0.01009156 0.00459208 0.0060921
 0.00719906 0.00354825 0.00611408 0.00675255 0.0057358  0.00544

[0.00271657 0.05405605 0.00523398 0.00468487 0.00641851 0.00421711
 0.00495178 0.00859226 0.10339894 0.00301678 0.00560442 0.00404615
 0.00732541 0.03430651 0.00797938 0.00692751 0.01469739 0.00913885
 0.00296113 0.00650274 0.01001049 0.01575901 0.00397162 0.00600528
 0.00968313 0.00773347 0.00882376 0.00855664 0.00743465 0.00485419
 0.00958079 0.0054883  0.00348934 0.00593611 0.00684789 0.03868607
 0.00313669 0.00495837 0.0051388  0.01121777 0.01013806 0.0071846
 0.01884307 0.02351496 0.01194744 0.00514722 0.0070136  0.00455136
 0.01021226 0.01277658 0.00949596 0.02197023 0.00429745 0.01083686
 0.00402169 0.00348047 0.00504386 0.00453805 0.00883406 0.00567301
 0.0055608  0.00788079 0.0065896  0.00833051 0.00709831 0.02105965
 0.01008647 0.0086389  0.00658943 0.02891779 0.00908148 0.00464565
 0.0053451  0.00961412 0.01126924 0.00844119 0.00459155 0.00562756
 0.00771216 0.00934297 0.00704867 0.01431539 0.00802619 0.00519964
 0.00589127 0.00238793 0.00861494 0.00861518 0.00525735 0.00530

[0.00411974 0.10112165 0.00399658 0.0033271  0.00431567 0.00344232
 0.00884555 0.0068011  0.08903778 0.00242545 0.01263801 0.0025774
 0.00495851 0.01959149 0.00403672 0.0045038  0.01693655 0.00422018
 0.00287098 0.00406653 0.00461652 0.00615703 0.00349918 0.00592533
 0.00415431 0.00781233 0.00755394 0.00480319 0.00882904 0.00485837
 0.00828596 0.00428064 0.00332466 0.00315649 0.00397788 0.04809298
 0.00292175 0.0037177  0.0043439  0.03768282 0.00644564 0.00467829
 0.03466122 0.03947658 0.00724975 0.00413978 0.00650428 0.00284683
 0.00940893 0.00969106 0.01006074 0.01327854 0.00251703 0.00476659
 0.00218671 0.00220688 0.00474198 0.00770452 0.00835317 0.00560183
 0.00734108 0.00416546 0.00208084 0.005682   0.0044614  0.03193132
 0.01418619 0.02087797 0.00478443 0.06199207 0.0069815  0.00330903
 0.00384548 0.0088829  0.00656468 0.00528177 0.00327259 0.00539577
 0.00411485 0.00649292 0.00525529 0.00824377 0.0063443  0.00301398
 0.0052135  0.00390732 0.00637182 0.00373031 0.00344316 0.00456

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover. We'll be using the formulas used in the classroom:

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [111]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculate portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    pd_weights = pd.DataFrame(all_rebalance_weights)
    n_rebalance_events_per_year = n_trading_days_in_year/shift_size
    
    df_weights_diff = pd_weights.diff(periods=-1).abs()
    series_weights_diff_col_sum = df_weights_diff.sum()
    sum_total_turnover = series_weights_diff_col_sum.sum()
    portfolio_turnover = (sum_total_turnover / rebalance_count) * n_rebalance_events_per_year
    
    #"OrderedDict([('portfolio_turnover', 80.0434875733)])"
    
    return portfolio_turnover

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)

Tests Passed


Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [112]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

16.726832615089062


That's it! You've built a smart beta portfolio in part 1 and did portfolio optimization in part 2. You can now submit your project.

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.